In [191]:


import glob 
svpath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
resultStrings = glob.glob(svpath+"savedBestSeed-18*")
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResults/TestFold/savedBestSeed-5*")
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResultsNN/sa*")
#skipsubj = True
skipSubjects = []
onlySignificant = True



In [192]:
#!pip3 install pandas -q

from array import ArrayType
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []



results = []
for path in resultStrings:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10 
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s , subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)
    
    currSeed = subject[1]
    currSubj = subject[2]
    
    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] +=1


    for aResults in subject[3]: 
        for aResults2 in aResults:
            #If the np array comes from svm results
            if len(aResults2)<6:
                
                aName = aResults2[0]
                ares = aResults2[1]
               
            
                if type(ares) is tuple:
                    ares = ares[0]
                    
                akernel = aResults2[2]
                aC = round(aResults2[3],2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1
        
                combNr = combinationList.index([nameDict[aName], kernelDict[akernel], aC])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:,0] == combNr)[0])
                    
                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])
                    
                else:
                  
                    combinationResult[currSubj].append([combNr, ares]) 
                    
                    #If the np array comes from NN results
            else:
               
                aName = aResults2[0]
                ares = aResults2[1]
 
                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3],2)
                areg = round(aResults2[4],2)
                adrp = round(aResults2[5],2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aLz, areg, adrp] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                    nrOfCombination += 1
                #combResult = []
                combNr = combinationList.index([nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:,0] == combNr)[0])
                    
                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])
                    
                else:
                    
                    combinationResult[currSubj].append([combNr, ares])



#Making table

headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]

subjectResultArray = np.empty([10, nrOfCombination],dtype=object) 
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination) 

for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < 0.55:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] +  res[1]
            comboTotal[0, res[0]] = comboAverage[0, res[0]] +  res[1]
            nrOfSubjects[res[0]] +=1
            subjectResultArray[subNr, res[0]] = res[1]

#Print the table!
import pandas as pd
comboAverage = comboAverage/nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(subjectResultArray) ,columns = headers)
display(table)




(1, 77)
(10, 77)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_24528\4103248225.py:140: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage/nrOfSubjects


,0:0:0.5:,0:1:0.5:,0:1:1.62:,0:1:2.75:,0:1:3.88:,0:1:5.0:,0:2:0.5:,0:2:1.62:,0:2:2.75:,0:2:3.88:,0:2:5.0:,1:0:0.5:,1:1:0.5:,1:1:1.62:,1:1:2.75:,1:1:3.88:,1:1:5.0:,1:2:0.5:,1:2:1.62:,1:2:2.75:,1:2:3.88:,1:2:5.0:,2:0:0.5:,2:1:0.5:,2:1:1.62:,2:1:2.75:,2:1:3.88:,2:1:5.0:,2:2:0.5:,2:2:1.62:,2:2:2.75:,2:2:3.88:,2:2:5.0:,3:0:0.5:,3:1:0.5:,3:1:1.62:,3:1:2.75:,3:1:3.88:,3:1:5.0:,3:2:0.5:,3:2:1.62:,3:2:2.75:,3:2:3.88:,3:2:5.0:,4:0:0.5:,4:1:0.5:,4:1:1.62:,4:1:2.75:,4:1:3.88:,4:1:5.0:,4:2:0.5:,4:2:1.62:,4:2:2.75:,4:2:3.88:,4:2:5.0:,5:0:0.5:,5:1:0.5:,5:1:1.62:,5:1:2.75:,5:1:3.88:,5:1:5.0:,5:2:0.5:,5:2:1.62:,5:2:2.75:,5:2:3.88:,5:2:5.0:,6:0:0.5:,6:1:0.5:,6:1:1.62:,6:1:2.75:,6:1:3.88:,6:1:5.0:,6:2:0.5:,6:2:1.62:,6:2:2.75:,6:2:3.88:,6:2:5.0:
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.595335,None,None,None,None,None,None,0.831579,0.832177,0.878349,0.876555,0.75,0.7,0.7,0.7,0.75,0.75,0.7,0.6,None,None,None,0.8,0.65,0.75,0.8,0.75,0.75,0.55,0.8,0.75,0.65,0.6,0.75,0.65,0.75,0.75,0.8,0.8,0.75,0.75,0.6,0.6,0.6,0.75,0.7,0.7,0.7,0.75,0.75,0.7,0.65,0.75,0.65,None,0.75,0.65,0.55,0.8,0.8,0.75,None,0.75,0.8,0.8,0.75,0.75,0.65,0.7,0.75,0.75,0.8,0.55,0.8,0.75,0.55,0.55
2,None,None,None,0.583333,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,0.666667,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,None,0.625,0.708333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625
3,0.611111,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.555556,None,None,0.555556,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,0.666667,None,None,None,None,None,None,None,None,None,None,0.625,None,None,None,None,None,None,None,None,0.625,0.625,0.75,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,0.583333,0.583333,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None
6,None,None,None,0.590909,0.590909,0.636364,None,None,None,None,None,0.590909,None,0.590909,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.590909,None,None,0.590909,None,None,None,None,None,None,0.590909,None,0.636364,None,None,None,None,None,None,None,None,None,None,0.590909,None,None,None,None,None,None,None,None,None,None,0.590909,None,None,None,None,None,None,None,None
7,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None
8,0.6,None,0.6,0.6,0.65,None,None,None,Non

In [193]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(comboAverage) ,columns = headers)
display(table)


print(f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}")
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))
maxInd = int(np.where(comboAverage == maxVal)[1])
print(maxInd)
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")

[0.         0.71839387 0.625      0.57407407 0.625      0.58333333
 0.59917355 0.58333333 0.59583333 0.58333333]


,0:0:0.5:,0:1:0.5:,0:1:1.62:,0:1:2.75:,0:1:3.88:,0:1:5.0:,0:2:0.5:,0:2:1.62:,0:2:2.75:,0:2:3.88:,0:2:5.0:,1:0:0.5:,1:1:0.5:,1:1:1.62:,1:1:2.75:,1:1:3.88:,1:1:5.0:,1:2:0.5:,1:2:1.62:,1:2:2.75:,1:2:3.88:,1:2:5.0:,2:0:0.5:,2:1:0.5:,2:1:1.62:,2:1:2.75:,2:1:3.88:,2:1:5.0:,2:2:0.5:,2:2:1.62:,2:2:2.75:,2:2:3.88:,2:2:5.0:,3:0:0.5:,3:1:0.5:,3:1:1.62:,3:1:2.75:,3:1:3.88:,3:1:5.0:,3:2:0.5:,3:2:1.62:,3:2:2.75:,3:2:3.88:,3:2:5.0:,4:0:0.5:,4:1:0.5:,4:1:1.62:,4:1:2.75:,4:1:3.88:,4:1:5.0:,4:2:0.5:,4:2:1.62:,4:2:2.75:,4:2:3.88:,4:2:5.0:,5:0:0.5:,5:1:0.5:,5:1:1.62:,5:1:2.75:,5:1:3.88:,5:1:5.0:,5:2:0.5:,5:2:1.62:,5:2:2.75:,5:2:3.88:,5:2:5.0:,6:0:0.5:,6:1:0.5:,6:1:1.62:,6:1:2.75:,6:1:3.88:,6:1:5.0:,6:2:0.5:,6:2:1.62:,6:2:2.75:,6:2:3.88:,6:2:5.0:
0,0.597445,NaN,0.6,0.591414,0.620455,0.609848,NaN,0.831579,0.832177,0.878349,0.729944,0.670455,0.7,0.645455,0.7,0.6875,0.708333,0.7,0.633333,0.583333,NaN,0.55,0.8,0.65,0.666667,0.8,0.75,0.6875,0.55,0.725,0.62963,0.65,0.6,0.62963,0.620455,0.75,0.6875,0.67197,0.775,0.75,0.75,0.575,0.6,0.6,0.670455,0.7,0.639899,0.625,0.75,0.75,0.7,0.619444,0.75,0.625,0.708333,0.75,0.65,0.570455,0.8,0.8,0.75,NaN,0.75,0.8,0.7,0.666667,0.666667,0.65,0.645455,0.75,0.666667,0.655556,0.55,0.8,0.75,0.55,0.5875


Max average accuracy 0.878349282296651
9
dataWCV
sigmoid
C: 3.88


In [194]:
table = pd.DataFrame(np.array(comboTotal) ,columns = headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal)==False)]))
maxInd2 = int(np.where(comboTotal == maxVal2)[1])
print(f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} ")

print(list(nameDict.keys())[combinationList[maxInd2][0]])
print(list(kernelDict.keys())[combinationList[maxInd2][1]])
print(f"C: {combinationList[maxInd2][2]} ")

,0:0:0.5:,0:1:0.5:,0:1:1.62:,0:1:2.75:,0:1:3.88:,0:1:5.0:,0:2:0.5:,0:2:1.62:,0:2:2.75:,0:2:3.88:,0:2:5.0:,1:0:0.5:,1:1:0.5:,1:1:1.62:,1:1:2.75:,1:1:3.88:,1:1:5.0:,1:2:0.5:,1:2:1.62:,1:2:2.75:,1:2:3.88:,1:2:5.0:,2:0:0.5:,2:1:0.5:,2:1:1.62:,2:1:2.75:,2:1:3.88:,2:1:5.0:,2:2:0.5:,2:2:1.62:,2:2:2.75:,2:2:3.88:,2:2:5.0:,3:0:0.5:,3:1:0.5:,3:1:1.62:,3:1:2.75:,3:1:3.88:,3:1:5.0:,3:2:0.5:,3:2:1.62:,3:2:2.75:,3:2:3.88:,3:2:5.0:,4:0:0.5:,4:1:0.5:,4:1:1.62:,4:1:2.75:,4:1:3.88:,4:1:5.0:,4:2:0.5:,4:2:1.62:,4:2:2.75:,4:2:3.88:,4:2:5.0:,5:0:0.5:,5:1:0.5:,5:1:1.62:,5:1:2.75:,5:1:3.88:,5:1:5.0:,5:2:0.5:,5:2:1.62:,5:2:2.75:,5:2:3.88:,5:2:5.0:,6:0:0.5:,6:1:0.5:,6:1:1.62:,6:1:2.75:,6:1:3.88:,6:1:5.0:,6:2:0.5:,6:2:1.62:,6:2:2.75:,6:2:3.88:,6:2:5.0:
0,2.989779,0.0,1.2,2.374242,1.890909,1.80303,0.0,1.663158,1.664354,1.756699,2.043222,1.931818,1.4,1.881818,1.4,2.0,2.083333,1.4,1.933333,1.166667,0.0,1.1,1.6,1.3,1.916667,1.6,1.5,2.0,1.1,2.1,2.472222,1.3,1.2,2.472222,1.831818,1.5,2.0,2.606818,2.3,1.5,1.5,1.7,1.8,1.8,1.931818,1.4,2.556061,1.8,1.5,1.5,1.4,2.441667,1.5,2.475,1.416667,1.5,1.3,1.731818,1.6,1.6,1.5,0.0,1.5,1.6,2.0,1.916667,1.916667,1.3,1.881818,1.5,1.916667,2.55,1.1,1.6,1.5,1.1,1.8


Max total significant accuracy then averaged 0.7474448431685273 
dataWCV
linear
C: 0.5 


In [195]:


maxAcc = np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])])
print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
print(np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])]))

Max single accuracy position (array([1], dtype=int64), array([9], dtype=int64))
0.878349282296651
